In [1]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt

from scipy.signal import find_peaks
from scipy.ndimage import uniform_filter1d
from tqdm import tqdm
from julian import julian
%matplotlib widget

In [2]:
start = dt.datetime(2013,1,1); stop = dt.datetime(2013,2,1); 
sat = 'GOCE';

In [3]:
# filepath = "/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI_Project_1/Data/GOCE/"
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\GOCE\\'
os.chdir(filepath)
filename = "goce_denswind_v1_5_2013-01.txt"
lines = np.loadtxt(filename,comments="#",dtype='str')

In [4]:
time_array = [];
Data= np.zeros((len(lines),15))
for i,l in enumerate(lines):
    time_array.append(dt.datetime.strptime(l[0] + ' ' +l[1], '%Y-%m-%d %H:%M:%S.%f'))
    for j in range(3,18):
        Data[i,j-3] = float(l[j])
columns = ['alt','lon','lat','lst','arglat','dens_x_it','cr_wnd_e','cr_wnd_n','cr_wnd_u','denserror','winderro','no','ec', 'as','th']
df1 = pd.DataFrame(Data,index=pd.DatetimeIndex(time_array),columns=columns)
# Data = Data.loc[start:stop]

In [5]:
%load_ext julia.magic
%julia using Dates;
%julia using LinearAlgebra;
%julia using DataFrames;
%julia using SatelliteToolbox;
%julia init_space_indices();
#dtcfile_force_download = true,fluxtable_force_download = true,solfsmy_force_download=true,wdcfiles_force_download = true,wdcfiles_oldest_year = 2000,wdcfiles_newest_year = 2010

Initializing Julia interpreter. This may take some time...


In [6]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\EPB stuff\\Data\\OMNI Data\\'
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/EPB stuff/Data/OMNI Data/';
os.chdir(filepath)
filename = 'omni2_hourly_indices.txt';
SW_Data_hr = pd.DataFrame(np.loadtxt(filename,dtype = float));
time_array = [dt.datetime(int(SW_Data_hr.iloc[i,0]),1,1) + dt.timedelta(days = int(SW_Data_hr.iloc[i,1])-1) + dt.timedelta(hours= int(SW_Data_hr.iloc[i,2])) for i in range(len(SW_Data_hr))];
SW_Data_hr.index = pd.DatetimeIndex(time_array);
columns = ['Year','DOY','Hour','B0','Bx','By','Bz','Plasma Speed','Flow Pressure','Electric Field','Kp','R','Dst','AP','F107','AE','AL','AU'];
SW_Data_hr.columns = columns;
SW_Data_hr = SW_Data_hr.loc[dt.datetime(start.year-1,1,1):dt.datetime(start.year+1,1,1)]

In [7]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\OMNI\\';
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI_Project_1/Data/OMNI/';
os.chdir(filepath)
filename = 'noaa_radio_flux.csv'
flux = pd.read_csv(filename)
time_array = [dt.datetime(int(str(flux.iloc[i,0])[:4]),int(str(flux.iloc[i,0])[4:6]),int(str(flux.iloc[i,0])[6:])) for i in range(len(flux))]
flux.index = pd.DatetimeIndex(time_array)
flux = flux.loc[dt.datetime(start.year-1,1,1):dt.datetime(start.year+1,1,1)]

In [8]:
df1

,alt,lon,lat,lst,arglat,dens_x_it,cr_wnd_e,cr_wnd_n,cr_wnd_u,denserror,winderro,no,ec,as,th
2013-01-01 00:00:00,244567.566,108.658,0.665,7.182,179.271,5.060560e-11,-50.75,8.96,0.00,5.079330e-13,9.03,0.0,0.0,1.0,0.0
2013-01-01 00:00:10,244710.888,108.540,-0.006,7.177,179.942,5.016900e-11,-52.50,9.28,0.00,5.033340e-13,9.11,0.0,0.0,1.0,0.0
2013-01-01 00:00:20,244859.272,108.421,-0.678,7.172,180.614,4.957630e-11,-55.89,9.88,-0.00,4.971820e-13,9.22,0.0,0.0,1.0,0.0
2013-01-01 00:00:30,245012.696,108.302,-1.349,7.166,181.285,4.901510e-11,-56.08,9.93,-0.00,4.912000e-13,9.32,0.0,0.0,1.0,0.0
2013-01-01 00:00:40,245171.133,108.183,-2.020,7.161,181.956,4.866480e-11,-56.43,10.02,-0.00,4.876350e-13,9.38,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-01-31 09:51:20,254884.387,104.857,79.037,16.617,81.095,2.949090e-11,113.83,86.87,0.11,3.065500e-13,39.94,0.0,1.0,0.0,0.0
2013-01-31 09:51:30,254972.168,102.602,79.565,16.470,81.768,2.904310e-11,112.39,92.48,0.11,3.026470e-13,40.58,0.0,1.0,0.0,0.0
2013-01-31 09:51:40,255056.411,100.119,80.075,16.307,82.440,2.871460e-11,108.95,97.32,0.11,2.993110e-13,41.01,0.0,1.0,0.0,0.0
2013-01-31 09:51:50,255137.082,97.379,80.567,16.127,83.113,2.829270e-11,103.80,101.46,0.11,2.947150e-13,41.58,0.0,1.0,0.0,0.0


In [9]:
N = len(df1)
inputs = np.zeros((N,6)); AP = np.zeros((N,7));
for i in tqdm(range(N)):
    t = df1.index[i]
    jd = t.to_julian_date()
    t1 = julian.from_jd(np.ceil(jd))
    inputs[i,0] = jd
    inputs[i,1] = df1['alt'][i] ## in meter
    inputs[i,2] = df1['lat'][i] ## in deg
    inputs[i,3] = df1['lon'][i] ## in deg
    
    idx = np.where((flux.index==dt.datetime(t.year,t.month,t.day)))[0][0]
    inputs[i,4] = flux['f107_observed (solar flux unit (SFU))'][idx-1]
    
    idxa = np.where((flux.index >= t - dt.timedelta(days=45)) & (flux.index <= t + dt.timedelta(days=45)))[0]-1
    inputs[i,5] = round(np.mean(flux['f107_observed (solar flux unit (SFU))'][idxa]),1)
    
    idx = np.where((SW_Data_hr['Year']==t.year)&(SW_Data_hr['DOY']==t.timetuple().tm_yday))[0]
    AP[i,0] = round(SW_Data_hr['AP'][idx].mean(),1)
    
    idx = np.where((SW_Data_hr['Year']==t.year)&(SW_Data_hr['DOY']==t.timetuple().tm_yday)&(SW_Data_hr['Hour']==t.hour))[0][0]
    AP[i,1] = SW_Data_hr['AP'][idx] ## 3 hour Ap index at the time
    AP[i,2] = SW_Data_hr['AP'][idx-3]  ## 3 hour Ap index 3 hrs before the time
    AP[i,3] = SW_Data_hr['AP'][idx-6]  ## 3 hour Ap index 6 hrs before the time
    AP[i,4] = SW_Data_hr['AP'][idx-9]
    AP[i,5]=SW_Data_hr['AP'][np.arange(idx-33,idx-12+1,3)].mean()
    AP[i,6]= SW_Data_hr['AP'][np.arange(idx-57,idx-36+1,3)].mean()

100%|██████████| 251895/251895 [14:27<00:00, 290.35it/s]


In [10]:
## calculate earth radius at a given lat
def radius(B):
    B = np.deg2rad(B);
    a = 6.378137 * 10**6;
    b = 6.356752 * 10**6;
    c = ((a**2)*np.cos(B))**2
    d = ((b**2)*np.sin(B))**2
    e = (a*np.cos(B))**2
    f = (b*np.sin(B))**2
    R = np.sqrt((c+d)/(e+f))
    return R
# R_e = [radius(inputs[i,2]) for i in range(len(df1))];
dist= [radius(inputs[i,2]) + inputs[i,1] for i in range(len(df1))]

## At satellite altitude

In [11]:
%julia inputs=$inputs; AP=$AP;N= size(inputs)[1]; Data=[];
# %julia gd = [geocentric_to_geodetic(deg2rad(inputs[i,3]),dist[i]) for i=1:N];
%julia for i in 1:N;push!(Data,nrlmsise00(inputs[i,1],inputs[i,2],deg2rad(inputs[i,3]),deg2rad(inputs[i,4]),inputs[i,6],inputs[i,5],AP[i,:],output_si=true,dversion=false)); end;
%julia Data_MSIS = DataFrame(Data);

In [12]:
rho = %julia Data_MSIS.den_Total
nO2 = %julia Data_MSIS.den_O2
nN2 = %julia Data_MSIS.den_N2
nO =  %julia Data_MSIS.den_O
# nAO = %julia Data_MSIS.den_aO
nAr = %julia Data_MSIS.den_Ar
nN =  %julia Data_MSIS.den_N
nHe = %julia Data_MSIS.den_He
nH =  %julia Data_MSIS.den_H
T_exo=%julia Data_MSIS.T_exo
Tz =  %julia Data_MSIS.T_alt
# gd = %julia gd

In [13]:
Data_MSIS = pd.DataFrame({'rho': rho,
                       'nO2': nO2,
                        'nN2': nN2,
                        'nO' : nO,
                        'nN':nN,
                        'nHe':nHe,
                        'nH':nH,
                          'nAr':nAr,
                        'T_exo':T_exo,
                        'Tz':Tz
                       },index = df1.index)

In [14]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\Temp\\';
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI_Project_1/Data/OMNI/';
os.chdir(filepath)
Data_MSIS.to_csv(sat+'_MSIS_jl_'+start.strftime("%Y-%m-%d")+'.csv')

## At 260 km

In [15]:
%julia inputs=$inputs; AP=$AP;N= size(inputs)[1]; Data=[];
# %julia gd = [geocentric_to_geodetic(deg2rad(inputs[i,3]),dist[i]) for i=1:N];
%julia for i in 1:N;push!(Data,nrlmsise00(inputs[i,1],260e3,deg2rad(inputs[i,3]),deg2rad(inputs[i,4]),inputs[i,6],inputs[i,5],AP[i,:],output_si=true,dversion=false)); end;
%julia Data_MSIS = DataFrame(Data);

In [16]:
rho = %julia Data_MSIS.den_Total
nO2 = %julia Data_MSIS.den_O2
nN2 = %julia Data_MSIS.den_N2
nO =  %julia Data_MSIS.den_O
# nAO = %julia Data_MSIS.den_aO
nAr = %julia Data_MSIS.den_Ar
nN =  %julia Data_MSIS.den_N
nHe = %julia Data_MSIS.den_He
nH =  %julia Data_MSIS.den_H
T_exo=%julia Data_MSIS.T_exo
Tz =  %julia Data_MSIS.T_alt
# gd = %julia gd

In [17]:
Data_MSIS = pd.DataFrame({'rho': rho,
                       'nO2': nO2,
                        'nN2': nN2,
                        'nO' : nO,
                        'nN':nN,
                        'nHe':nHe,
                        'nH':nH,
                          'nAr':nAr,
                        'T_exo':T_exo,
                        'Tz':Tz
                       },index = df1.index)

In [18]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI_Project_1\\Data\\Temp\\';
# filepath = '/Users/user/OneDrive - The University of Texas at Dallas/Documents/MURI_Project_1/Data/OMNI/';
os.chdir(filepath)
Data_MSIS.to_csv(sat+'_MSIS_jl_260_km_'+start.strftime("%Y-%m-%d")+'.csv')